In [ ]:
import os

# we will definitely need pyConText
import pyConTextNLP
from pyConTextNLP import pyConTextGraph
from pyConTextNLP.itemData import itemData
from pyConTextNLP.display.html import mark_document_with_html
print(pyConTextNLP.__version__)
# useful utilities in RadNLP as well
import radnlp
import radnlp.view as rview
from radnlp.data import classrslts
# we will need a few other packages
from textblob import TextBlob
import urllib
import pandas as pd
# packages for interaction
from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display, HTML, Image
import ipywidgets
# and also our utilities for this class
from nlp_pneumonia_utils import Annotation
from nlp_pneumonia_utils import AnnotatedDocument
from nlp_pneumonia_utils import read_brat_annotations
from nlp_pneumonia_utils import read_doc_annotations
from nlp_pneumonia_utils import read_annotations
from nlp_pneumonia_utils import calculate_prediction_metrics
from nlp_pneumonia_utils import mark_text
from nlp_pneumonia_utils import pneumonia_html_markup
from nlp_pneumonia_utils import clearPyConTextRegularExpressions

print('Imported pneumonia nlp utilities...')

# First thing, let's load our training set

In [ ]:
annotated_doc_map = read_doc_annotations('data/training.zip')
#annotated_doc_map = read_doc_annotations('pneumonia_brat_full_set1.zip')
# let's also use a simple list of documents as well as this map
annotated_docs = list(annotated_doc_map.values())

print('Total Annotated Documents : {0}'.format(len(annotated_docs)))

In [ ]:
def markup_sentence(s, modifiers, targets, prune_inactive=True, verbose = False):
    """
    """
    markup = pyConTextGraph.ConTextMarkup()
    markup.setRawText(s)
    markup.cleanText()
    markup.markItems(targets, mode="target")
    markup.markItems(modifiers, mode="modifier")
    markup.pruneMarks()
    markup.dropMarks('Exclusion')
    # apply modifiers to any targets within the modifiers scope
    markup.applyModifiers()
    markup.pruneSelfModifyingRelationships()
    if prune_inactive:
        markup.dropInactiveModifiers()
    return markup

In [ ]:
# let us set up an example document to work with
example_document = """
PORTABLE CHEST:  Comparison made to prior film from X:XX a.m. the same day.
     
The ET tube and nasogastric tube remain in good position. Cardiac and
mediastinal contours are stable. No acute changes are seen within the lung
parenchyma; specifically, there is no evidence of new infiltrate (skin folds
do project over the right lung). No consolidation on either side.

IMPRESSION: No evidence of pneumonia."""

example_sentence = """IMPRESSION: No evidence of pneumonia."""

# Before we continue, note that any itemData in pyConText has 4 parts:
1. The literal (e.g. "pneumonia", "pneumoniathorax", "can rule out", "cannot be excluded", etc)
2. The category (e.g. "EVIDENCE_OF_PNEUMONIA")
3. The regular expression (optional) used to capture the literal in the text. If no regular expression is provided, a regular expression is generated literally from the literal.
4. The rule (optional). If the itemData is being used as a modifier, the rule states what direction the modifier operates in the sentence: current valid values are: "forward", the item can modify objects following it in the sentence; "backward", the item can modify objects preceding it in the sentence; or "bidirectional", the item can modify objects preceding and following it in the sentence.

In [ ]:
# Now let's set up some rules for pyConText for EVIDENCE_OF_PNEUMONIA
# At this moment, we will just set up these "concepts" and well handle modifiers for them after that

targets1 = []
modifiers1 = []

# so before we add targets, remember from above that they will look like this : 
# targets = itemData(["literal", "CATEGORY", "regular expression(s)", "empty or forward or backward or bidirectional"])

# so now let's set this up for "pneumonia" with the category "EVIDENCE_OF_PNEUMONIA"
targets1 = itemData(["pneumonia", "EVIDENCE_OF_PNEUMONIA", "", ""])

# let's go ahead and use this now on one single example sentence:
markup = markup_sentence(example_sentence, modifiers1, targets1)
# prettier display with IPython display
display(markup.nodes(data = True))
#print(markup.getXML())

In [ ]:
# This function now works on entire documents combining all sentence-level objects into one object we can can then graph
def markup_context_document(report_text, modifiers, targets):
    context = pyConTextGraph.ConTextDocument()
    
    # we will use TextBlob for breaking up sentences
    sentences = [s.raw for s in TextBlob(report_text).sentences]
    for sentence in sentences:
        m = markup_sentence(sentence, modifiers=modifiers, targets=targets)
        context.addMarkup(m)
    
    return context

In [ ]:
example_sentence_2 = """Findings consistent with CHF, although underlying bilateral lower lobe pneumonias cannot be excluded."""

In [ ]:
# let's see how things look on this sentence
markup_sentence_2 = markup_sentence(example_sentence_2, modifiers1, targets1, verbose = True)
display(markup_sentence_2.nodes(data = True))

# So we didn't mark up a target for "pneumonias" since we only had the singular variant "pneumonia".  Let's add that as we augment our target concepts

In [ ]:
pneumonia_targets_file = 'KB/pneumonia_targets.tsv'

# let's see what we're working with by loading this as a Pandas DataFrame and then we can display it
pneumonia_targets_df = pd.read_csv(pneumonia_targets_file, delimiter = '\t')
display(pneumonia_targets_df)

In [ ]:
# Our first attempt was very simple target, so now let's add some additional concepts
targets2 = []
modifiers2 = []

# so before we add targets, remember from above that they will look like this : 
# targets = itemData(["literal", "CATEGORY", "regular expression(s)", "empty or forward or backward or bidirectional"])

# before we continue, let's clear a mapping of compiled regular expressions which pyConText uses
clearPyConTextRegularExpressions()

# so now let's set this up with more variants of "EVIDENCE_OF_PNEUMONIA"
full_targets_path = 'file:///' + os.path.join(os.getcwd(), pneumonia_targets_file)
print('Loading pneumonia targets from : ' + full_targets_path)
targets2 = pyConTextNLP.itemData.instantiateFromCSVtoitemData(full_targets_path)

# let's go ahead and use this again on our updated targets
context = markup_context_document(example_document, modifiers2, targets2)
# prettier display with IPython display
display(context.getDocumentGraph().nodes(data = True))
#print(context.getXML())

## Let's look at this markup in HTML with colors

In [ ]:
evidence_only_colors = {
    "evidence_of_pneumonia": "orange"
}

context_html = pyConTextNLP.display.html.mark_document_with_html(context, colors = evidence_only_colors, default_color="black")
display(HTML(context_html))

## Let's also look again to see if our regular expression for "pneumonia" and "pneumonias" worked properly

In [ ]:
markup_sentence_2_check = markup_sentence(example_sentence_2, modifiers2, targets2)
print(targets2)
display(markup_sentence_2_check.nodes(data = True))

# We've  added some pyConText targets, so let's utilize them in a classifier so that we can see that adding targets can increase our Recall even if Precision suffers
## We will address Precision when we start working with ConText Modifiers

In [ ]:
class ConTextTargetOnlyClassifier(object):
    def __init__(self, modifiers, targets):
        self.modifiers = modifiers
        self.targets = targets
    def predict(self, text):
        # let's use our other functions in this notebook to perform sentence-wise markup and
        # we can then check to see if these contain any EVIDENCE_OF_PNEUMONIA category types
        context = markup_context_document(text, self.modifiers, self.targets)
        document_graph = context.getDocumentGraph()
        
        # let's walk through all of the nodes in the graph and see how many are evidence of pneumonia
        pneumonia_evidence_count = 0
        for node in document_graph.nodes():
            category_list = node.getCategory()
            for category in category_list:
                if category.upper() == 'EVIDENCE_OF_PNEUMONIA':
                    pneumonia_evidence_count += 1
            
        # do we have at least one category of pneumonia evidence here?
        return (pneumonia_evidence_count) > 0
           
# this one has only one target
classifier1 = ConTextTargetOnlyClassifier(modifiers1, targets1)
# this one has 3...
classifier2 = ConTextTargetOnlyClassifier(modifiers2, targets2)

# and now we can assess their performance
print('****************')
print('Performance for Classifier 1 : One total Target')
calculate_prediction_metrics(annotated_docs, classifier1.predict)

print('****************')
print('Performance for Classifier 2 : 3 total Targets')
calculate_prediction_metrics(annotated_docs, classifier2.predict)

# So we have improved recall, but what are we going to do about Precision  Since both Precision and Recall are measured equally in our F1 measure, we need to address it
## The solution to this is to improve our classification pipeline with ConText Modifiers

# Modifiers are used to add objects which modify other tags or items in the sentence.  For example, we've already added targets for evidence of pneumonia.  We can now use modifiers to ensure that instead of treating each of these mentions as affirmed evidence, we can modify them by the context of other words in the sentence.
## Concretely if we had a sentence "No evidence of pneumonia" our current pipelines would count this as a positive case of pneumonia when it is in fact the contrary.
## We can add a rule "no evidence of" to modify all targets which follow it
## Modifiers can also modify targets which came before it in a sentence such as "pneumonia will be ruled out"
## Modifiers can have the following values:
1. backward (modify any markup preceding it)
2. forward (modify any markup following it)
3. bidirectional (modify any markup following or preceding it

# Let's set up some demonstrations of how this works:

In [ ]:
probable_sentence_1 = """probable case of pneumonia"""
probable_sentence_2 = """no evidence of pneumonia and probable arthritis"""

In [ ]:
def view_single_sentence_graph(sentence, modifiers, targets):
    context = markup_context_document(sentence, modifiers, targets)
    class_result = classrslts(context_document=context, exam_type="Chest X-Ray", report_text=sentence, classification_result='N/A')
    rview.markup_to_pydot(class_result)
    display(Image("tmp.png"))
    print(sentence)
    
modifiers_forward = itemData(["probable", "PROBABLE_EXISTENCE", "", "forward"])
modifiers_backward = itemData(["probable", "PROBABLE_EXISTENCE", "", "backward"])

# Forward in this sentence modifies what we would expect

In [ ]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
clearPyConTextRegularExpressions()
view_single_sentence_graph(probable_sentence_1, modifiers_forward, targets1)

# In this example, nothing precedes "probable" so nothing to modify

In [ ]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
clearPyConTextRegularExpressions()
view_single_sentence_graph(probable_sentence_1, modifiers_backward, targets1)

# In this example, the forward rule does not pick up pneumonia since it comes before and we've not set up a target for arthritis

In [ ]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
clearPyConTextRegularExpressions()
view_single_sentence_graph(probable_sentence_2, modifiers_forward, targets1)

# And in this example, when we allow the target to work backward, it picks up a context where there seems to be "no evidence of pneumonia"

In [ ]:
# let's see how we graph the relationships between targets and modifiers based on 'backward' or 'forward':
clearPyConTextRegularExpressions()
view_single_sentence_graph(probable_sentence_2, modifiers_backward, targets1)

# Identifying the proper syntax for rules helps us implement the proper directionality of what it modifies

# There is another important attribute that Modifiers can employ : "terminate"
## This allows any modifier working forward or backward to stop its modifications if it encounters one of these terms.  Let's demonstrate an example where we want "probable" to modify "arthritis" as a condition but not "pneumonia":

In [ ]:
terminate_example_sentence = """probable arthritis but not pneumonia"""

In [ ]:
temp_targets = itemData(["pneumonia", "EVIDENCE_OF_PNEUMONIA", "", ""],
                       ["arthritis", "ANOTHER_CONDITION", "", "forward"])

modifiers_without_terminate = itemData(["probable", "PROBABLE_EXISTENCE", "", "forward"])

modifiers_with_terminate = itemData(["probable", "PROBABLE_EXISTENCE", "", "forward"],
                                   ["but", "CONJ", "", "terminate"])

# Without the "terminate" modifier "probable_existence" is applied to both arthritis and also pneumonia

In [ ]:
clearPyConTextRegularExpressions()
view_single_sentence_graph(terminate_example_sentence, modifiers_without_terminate, temp_targets)

# With the "terminate" modifier "probable_existence" is applied only to "arthritis" and does not modify beyond the conjunction "but"

In [ ]:
clearPyConTextRegularExpressions()
view_single_sentence_graph(terminate_example_sentence, modifiers_with_terminate, temp_targets)

## Developing modifiers takes time and objective measure.  Luckily, many of them have already been developed by Dr. Wendy Chapman and others on various research efforts.  Let's see what kind of data they contain

In [ ]:
context_modifiers_url = "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.tsv"

In [ ]:
modifier_file = urllib.request.urlopen(context_modifiers_url, data=None)
# now let's load this in directly into a DataFrame with Pandas and take a look at it
modifier_df = pd.read_csv(modifier_file, delimiter = "\t")
display(modifier_df.head(10))
display(modifier_df.tail(10))

In [ ]:
modifiers3 = pyConTextNLP.itemData.instantiateFromCSVtoitemData(context_modifiers_url)
# let's just use the same targets as above for our third pipeline
targets3 = targets2

clearPyConTextRegularExpressions()

print('Total Modifiers Loaded for pipeline #3 : [{0}]'.format(len(modifiers3)))
print('Total Targets Loaded for pipeline #3 : [{0}]'.format(len(targets3)))

# Now we can use leverage both Targets and Modifiers to properly leverage context let's see what this looks like in HTML with our document:

In [ ]:
# prepare some colors for displaying any markup we might see
colors = {
    "evidence_of_pneumonia": "orange",
    "definite_negated_existence": "red",
    "probable_negated_existence": "indianred",
    "ambivalent_existence": "orange",
    "probable_existence": "forestgreen",
    "definite_existence": "green",
    "historical": "goldenrod",
    "indication": "pink",
    "acute": "golden"
}

# let's mark up a new context object for our pipeline#3
context3 = markup_context_document(example_document, modifiers3, targets3)

display(HTML(pyConTextNLP.display.html.mark_document_with_html(context3, colors = colors, default_color="black")))

In [ ]:
# now let's take a closer look at the XML to see how this is working behind the scenes
print(context3.getXML())

# OK, so now that we've got some decent Targets and Modifiers to start from, let's process all of the documents and then visualize the relationships between Targets and Modifiers for some of these documents

In [ ]:
%%time
# NOTE : This is a "magic" command to Jupyter to time the execution of this entire cell

report_results = []
print('Marking up all documents...')
for anno_doc in annotated_docs:
    report_context = markup_context_document(anno_doc.text, modifiers3, targets3)
    # package this up into a class that the RadNLP utilities can use
    results = classrslts(context_document=report_context, exam_type="Chest X-Ray", report_text=anno_doc.text, classification_result='N/A')
    report_results.append(results)
    
print('DONE Marking up all documents...')

In [ ]:
# This function let's us iterate through all documents and view the markup
def view_markup(class_results, colors):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(class_results)-1))
    def _view_markup(i):
        class_result = class_results[i]
        rview.markup_to_pydot(class_result)
        display(Image("tmp.png"))
        
        report_html = pyConTextNLP.display.html.mark_document_with_html(class_result.context_document, colors = evidence_only_colors, default_color="black")
        
        display(HTML(report_html))

In [ ]:
view_markup(report_results, colors)

# Now that we can see how modifiers can interact with Targets, let's see if we can use a toy example to see how this can help improve Precision

## NOTE : The simple classifier coded her below is not an example of how pyConText should truly be used.  Jianlin will talk about a cleaner way to set up classification rules, but until then we can use this approach to determine if we can improve Precision (i.e. reduce False Positives) by only classifying on 1 or more mentions of pneumonia evidence which are not negated.

In [ ]:
# so now that we have added some pyConText targets, let's wire this up into a classifier so that we 
# can see that adding targets can increase our Recall even if Precision suffers
# We will address Precision when we start working with ConText Modifiers
class ConTextTargetModifierToyClassifier(object):
    def __init__(self, modifiers, targets):
        self.modifiers = modifiers
        self.targets = targets
        # here is a list of modifiers which we do not want to allow for "positive" classification
        self.negative_modifier_set = set(['definite_negated_existence', 'probable_negated_existence'])
        
    def predict(self, text):
        # let's use our other functions in this notebook to perform sentence-wise markup and
        # we can then check to see if these contain any EVIDENCE_OF_PNEUMONIA category types
        context = markup_context_document(text, self.modifiers, self.targets)
        document_graph = context.getDocumentGraph()
        
        # NOTE : Not necessary to understand the code below as long as you understand that :
        # we're now only counting "positive pneumonia" as long as it's not modified by one of the negative modifiers
        # in "self.negative_modifier_set"
        pneumonia_evidence_count = 0
        # this returns tuples of (tagObject, dictionary)
        nodes = document_graph.nodes(data = True)
        nodes.sort()
        for node in nodes:
            tag_object = node[0]
            for category in tag_object.getCategory():
                if category.upper() == 'EVIDENCE_OF_PNEUMONIA':
                    # let's fetch any nodes which "modify" our evidence
                    modifier_category_set = set()
                    modified_by = document_graph.predecessors(tag_object)
                    if modified_by:
                        for modifier in modified_by:
                            modifier_category_set = modifier_category_set.union(set(modifier.getCategory()))
                    # let's see if this node was modified by any of the set that we currently do not allow
                    found_negative_modifiers = modifier_category_set.intersection(self.negative_modifier_set)
                    if len(found_negative_modifiers) == 0:
                        pneumonia_evidence_count += 1
            
        # do we have at least one category of pneumonia evidence here?
        return (pneumonia_evidence_count) > 0
    
# instantiate our "toy" classifier which considers targets and their modifiers
target_modifier_toy_classifier = ConTextTargetModifierToyClassifier(modifiers3, targets3)

print('****************')
print('Performance for Classifier 3 : Combining Targets and Modifiers')
calculate_prediction_metrics(annotated_docs, target_modifier_toy_classifier.predict)

# and let's compare this side-y-side with our previous classifier of targets-only
print('****************')
print('Performance for Classifier 1 : 3 total Targets')
calculate_prediction_metrics(annotated_docs, classifier2.predict)

# Improving our system
## Now that we have the tools of adding/editing Targets and Modifiers in pyConText we can continue our rounds of Error Analysis as Jianlin demonstrated to us previously (see "NLP_ErrorAnalysis" notebook) and we can examine cases of False Positives and False Negatives so that we can increase Precision and Recall respectively:

In [ ]:
def list_false_negatives(gold_docs, prediction_function):
    fn_docs={}
    for doc_name, gold_doc in gold_docs.items():
        gold_label=gold_doc.positive_label;
        pred_label = prediction_function(gold_doc.text)
        if gold_label==1 and pred_label==0:
            fn_docs[doc_name]=gold_doc            
    return fn_docs  

def list_false_positives(gold_docs, prediction_function):
    fn_docs={}
    for doc_name, gold_doc in gold_docs.items():
        gold_label=gold_doc.positive_label;
        pred_label = prediction_function(gold_doc.text)
        if gold_label==0 and pred_label==1:
            fn_docs[doc_name]=gold_doc            
    return fn_docs  

In [ ]:
%%time

# get our current set of false negatives and false positives if we use our simple toy classifier
# which uses targets and a simplified implementation of modifiers
current_false_negatives = list_false_negatives(annotated_doc_map, target_modifier_toy_classifier.predict)
current_false_positives = list_false_positives(annotated_doc_map, target_modifier_toy_classifier.predict)

# prepare each of these for visualization
fn_report_results = []
fp_report_results = []
print('Marking up False Negatives')
for anno_doc in current_false_negatives.values():
    report_context = markup_context_document(anno_doc.text, modifiers3, targets3)
    # package this up into a class that the RadNLP utilities can use
    results = classrslts(context_document=report_context, exam_type="Chest X-Ray", report_text=anno_doc.text, classification_result='N/A')
    fn_report_results.append(results)
    
print('Marking up False Positives')
for anno_doc in current_false_positives.values():
    report_context = markup_context_document(anno_doc.text, modifiers3, targets3)
    # package this up into a class that the RadNLP utilities can use
    results = classrslts(context_document=report_context, exam_type="Chest X-Ray", report_text=anno_doc.text, classification_result='N/A')
    fp_report_results.append(results)

print('Current total False Negatives : {0}'.format(len(current_false_negatives)))
print('Current total False Positives : {0}'.format(len(current_false_positives)))

In [ ]:
view_markup(fn_report_results, colors)
print('Display for current False Negatives:')
print('Current total False Negatives : {0}'.format(len(current_false_negatives)))

In [ ]:
view_markup(fp_report_results, colors)
print('Display for current False Positives:')
print('Current total False Positives : {0}'.format(len(current_false_positives)))